Based on the code shown in https://scikit-image.org/docs/stable/auto_examples/features_detection/plot_sift.html

In [117]:
import matplotlib.pyplot as plt
from skimage.feature import SIFT, match_descriptors, plot_matched_features
from skimage import transform
from PIL import Image
import numpy as np
import os
import cv2
from skimage.measure import ransac
from skimage.transform import ProjectiveTransform
from pathlib import Path
import pandas as pd
from scipy.ndimage import gaussian_filter
import gc


In [84]:
# Load the datafiles and make them grayscale
def load_image(path):
    return np.array(Image.open(path).convert('L'), dtype=np.float32)

In [85]:
def preprocess_image(img_path, target_size=None, sigma=1):
    """
    Standard preprocessing:
    - grayscale
    - resize (optional)
    - normalize (zero mean, unit variance)
    - gaussian smoothing
    """
    im = np.array(Image.open(img_path).convert('L'), dtype=np.float32)
    if target_size:
        im = np.array(Image.fromarray(im).resize(target_size, Image.BICUBIC), dtype=np.float32)
    im = (im - np.mean(im)) / (np.std(im) + 1e-8)
    return gaussian_filter(im, sigma=sigma)

In [86]:
# Used to acces the name of the folder we are working on : not necessary
print("Répertoire courant :", os.getcwd())


Répertoire courant : c:\Users\emmar\OneDrive\Polytech\I5\Semestre 9 - Oslo\Data Mining\Exercices\Day 5\Augmented_reality_system\src


In [87]:
# Extract SIFT keypoints and descriptors
def extract_sift_features(image_filename):
    image = preprocess_image(image_filename)
    sift = SIFT()
    sift.detect_and_extract(image)
    return sift.keypoints, sift.descriptors

In [88]:
# Visualize the matches
def match_and_plot(img1, img2, keypoints1, descriptors1, keypoints2, descriptors2, title):
    matches = match_descriptors(descriptors1, descriptors2, max_ratio=0.6, cross_check=True)

    
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    plt.gray()
    plot_matched_features(image0=img1, image1=img2, 
                          keypoints0=keypoints1, keypoints1=keypoints2,
                          matches=matches, ax=ax, 
                          keypoints_color='cyan',
                          only_matches=True)
    ax.axis('off')
    ax.set_title(title)
    plt.show()
    return matches

In [ ]:
# Compute matching accuracy using optional homography
def compute_matching_accuracy(keypoints1, keypoints2, matches, homography=None, threshold=5):
    """
    Compute the matching accuracy based on a homography.
    If no homography is provided, only the number of matches is printed.
    """
    if homography is None:
        print(f"No homography matrix to work with")
        return None

    if matches is None or len(matches) == 0:
        print("No matches to evaluate.")
        return 0
        

    correct = 0
    for i, j in matches:
        pt1 = np.array([*keypoints1[i], 1.0])  # homogeneous coordinates
        projected = homography @ pt1
        projected /= projected[2]  # normalize

        pt2 = keypoints2[j]
        error = np.linalg.norm(projected[:2] - pt2)

        if error < threshold:
            correct += 1

    accuracy = correct / len(matches) if matches.size > 0 else 0
    #print(f"Matching Accuracy: {accuracy:.2%} ({correct}/{len(matches)} correct matches)")
    return accuracy

In [112]:
def create_homography(keypoints1, keypoints2, descriptors1, descriptors2):
    """
    Create a homography matrix from source points to destination points.
    """
    # Convert descriptors to float32 if needed
    descriptors1 = descriptors1.astype(np.float32)
    descriptors2 = descriptors2.astype(np.float32)

    # Match descriptors using BFMatcher + Lowe ratio test
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    raw_matches = bf.knnMatch(descriptors1, descriptors2, k=2)

    good_matches = []
    for m, n in raw_matches:
        if m.distance < 0.6 * n.distance:
            good_matches.append((m.queryIdx, m.trainIdx))

    if len(good_matches) < 4:
        print(f"Not enough matches to estimate homography: {len(good_matches)} found")
        return None, None

    matches = np.array(good_matches, dtype=int)
    
    if matches is None:
        print("No matches found.")
        return None, None
    
    src = keypoints1[matches[:, 0]]  # points from image 1
    dst = keypoints2[matches[:, 1]]  # corresponding points in image 2

    model_robust, inliers = ransac(
        (src, dst),
        ProjectiveTransform,
        min_samples=4,
        residual_threshold=2,
        max_trials=1000
    )
    H = model_robust.params 
    return H, matches


In [91]:
def load_base(dataset_dir = '../data/images'):
    dataset_paths = sorted(Path(dataset_dir).rglob("*"))
    records = []

    for ref_path in dataset_paths:
        print(f"Currently working on {ref_path}")
        
        if ref_path.suffix.lower() not in [".jpg", ".jpeg", ".png", ".avif"]:
            continue
            
        img_ref = load_image(str(ref_path))
        keypoints_ref, descriptors_ref = extract_sift_features(str(ref_path))

        if len(keypoints_ref) == 0:
            print(f" Skipped {ref_path.name} due to missing keypoints")
            continue

        record = {
            "filename" : ref_path.name,
            "path": str(ref_path),
            "num_keypoints": len(keypoints_ref),
            "keypoints": keypoints_ref,
            "descriptors": descriptors_ref,
            "image_shape": img_ref.shape,
            "image": img_ref,
        }
        records.append(record)

    df = pd.DataFrame(records)
    return df

In [92]:
data = load_base()

Currently working on ..\data\images\city_hall
Currently working on ..\data\images\city_hall\city_hall1.jpg


Currently working on ..\data\images\city_hall\city_hall2.jpg
Currently working on ..\data\images\city_hall\city_hall3.jpeg
Currently working on ..\data\images\city_hall\city_hall4.jpg
Currently working on ..\data\images\fram
Currently working on ..\data\images\fram\fram1.png
Currently working on ..\data\images\fram\fram2.jpeg
Currently working on ..\data\images\fram\fram3.jpeg
Currently working on ..\data\images\fram\fram4.jpg
Currently working on ..\data\images\fram\fram5.avif
Currently working on ..\data\images\opera
Currently working on ..\data\images\opera\opera1.jpg
Currently working on ..\data\images\opera\opera2.jpg
Currently working on ..\data\images\opera\opera3.jpeg
Currently working on ..\data\images\opera\opera4.jpg
Currently working on ..\data\images\opera\opera5.jpg
Currently working on ..\data\images\palace
Currently working on ..\data\images\palace\palace1.jpg
Currently working on ..\data\images\palace\palace2.jpeg
Currently working on ..\data\images\palace\palace3.jpeg

In [99]:
def comparison_all (ref_path):
    ref_path = Path(ref_path)
    results_dict = []

    if ref_path.suffix.lower() not in [".jpg", ".jpeg", ".png", ".avif"]:
        pass
    
    print(f"Working on {ref_path}")

    img_ref = load_image(str(ref_path))
    keypoints_ref, descriptors_ref = extract_sift_features(str(ref_path))

    for _, files in data.iterrows():
        H1, matches = create_homography(files["keypoints"], keypoints_ref, files["descriptors"], descriptors_ref)
        accuracy = compute_matching_accuracy(files['keypoints'], keypoints_ref, matches, H1)
        if accuracy is not None:
            results_dict.append({
                "test_file": str(ref_path),
                "compared_file": files["path"],
                "accuracy": accuracy
            })
    return results_dict


In [106]:
results = comparison_all('../data/test/city_hall_test.jpg')
print(results)

Working on ..\data\test\city_hall_test.jpg
Not enough matches to estimate homography: 3 found
No homography matrix to work with
Not enough matches to estimate homography: 2 found
No homography matrix to work with


c:\Users\emmar\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\numeric.py:442: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(res, fill_value, casting='unsafe')
C:\Users\emmar\AppData\Local\Temp\ipykernel_5700\2372129772.py:19: RuntimeWarning: divide by zero encountered in divide
  projected /= projected[2]  # normalize
C:\Users\emmar\AppData\Local\Temp\ipykernel_5700\2372129772.py:19: RuntimeWarning: invalid value encountered in divide
  projected /= projected[2]  # normalize


Not enough matches to estimate homography: 3 found
No homography matrix to work with
Not enough matches to estimate homography: 0 found
No homography matrix to work with
Not enough matches to estimate homography: 3 found
No homography matrix to work with
Not enough matches to estimate homography: 0 found
No homography matrix to work with
Not enough matches to estimate homography: 0 found
No homography matrix to work with


c:\Users\emmar\AppData\Local\Programs\Python\Python313\Lib\site-packages\skimage\measure\fit.py:975: UserWarning: No inliers found. Model not fitted
  warn("No inliers found. Model not fitted")


AttributeError: 'NoneType' object has no attribute 'params'

In [132]:
def comparison_small(ref_path):
    filename = os.path.basename(ref_path)  # "city_hall_test.jpg"
    folder_name = filename.split('_')[0]  # "city_hall"

    # Extraire le préfixe du nom de fichier (avant le premier "_")
    data["folder_name"] = data["filename"].str.split("_").str[0]

    # Filtrer les lignes correspondant au dossier "city_hall"
    df_element = data[data["path"].str.contains(folder_name)]

    img_ref = load_image(str(ref_path))
    keypoints_ref, descriptors_ref = extract_sift_features(str(ref_path))

    for _, files in df_element.iterrows() : 
        H1, matches = create_homography(files["keypoints"], keypoints_ref, files["descriptors"], descriptors_ref)
        accuracy = compute_matching_accuracy(files['keypoints'], keypoints_ref, matches, H1)
        print(f'Accuracy between {ref_path} and {files['filename']} : {accuracy}')



In [ ]:
comparison_small("../data/test/city_hall_test.jpg")



Accuracy between ../data/test/city_hall_test.jpg and city_hall1.jpg : 0.058823529411764705
Not enough matches to estimate homography: 3 found
No homography matrix to work with
Accuracy between ../data/test/city_hall_test.jpg and city_hall2.jpg : None
Not enough matches to estimate homography: 2 found
No homography matrix to work with
Accuracy between ../data/test/city_hall_test.jpg and city_hall3.jpeg : None
Accuracy between ../data/test/city_hall_test.jpg and city_hall4.jpg : 0.4166666666666667


In [134]:
comparison_small("../data/test/fram_test.jpg")

Accuracy between ../data/test/fram_test.jpg and fram1.png : 0.6666666666666666


c:\Users\emmar\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\numeric.py:442: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(res, fill_value, casting='unsafe')


Accuracy between ../data/test/fram_test.jpg and fram2.jpeg : 0.5
Accuracy between ../data/test/fram_test.jpg and fram3.jpeg : 0.6666666666666666
Accuracy between ../data/test/fram_test.jpg and fram4.jpg : 0.6363636363636364
Accuracy between ../data/test/fram_test.jpg and fram5.avif : 0.5


In [135]:
comparison_small("../data/test/palace_test.jpg")

Accuracy between ../data/test/palace_test.jpg and palace1.jpg : 0.4
Accuracy between ../data/test/palace_test.jpg and palace2.jpeg : 0.2
Not enough matches to estimate homography: 0 found
No homography matrix to work with
Accuracy between ../data/test/palace_test.jpg and palace3.jpeg : None
Not enough matches to estimate homography: 1 found
No homography matrix to work with
Accuracy between ../data/test/palace_test.jpg and palace4.jpg : None
Accuracy between ../data/test/palace_test.jpg and palace5.jpg : 0.5714285714285714


In [136]:
comparison_small("../data/test/vigeland_test.avif")

Accuracy between ../data/test/vigeland_test.avif and vigeland1.jpg : 0.8269230769230769
Accuracy between ../data/test/vigeland_test.avif and vigeland2.jpg : 0.5714285714285714


c:\Users\emmar\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\numeric.py:442: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(res, fill_value, casting='unsafe')


Accuracy between ../data/test/vigeland_test.avif and vigeland3.jpg : 0.23076923076923078
Accuracy between ../data/test/vigeland_test.avif and vigeland4.png : 0.6666666666666666


In [137]:
comparison_small("../data/test/opera_test.avif")

KeyboardInterrupt: 